In [4]:
import pandas as pd

In [5]:
rules = pd.read_json("../data/arules.json")
rules["antecedents"] = rules["antecedents"].apply(lambda x: frozenset(x))
rules["consequents"] = rules["consequents"].apply(lambda x: frozenset(x))

In [6]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(IAmA),(AskReddit),0.217120,0.480390,0.127371,0.586641,1.221177,0.023069,1.257043
1,(fffffffuuuuuuuuuuuu),(IAmA),0.003648,0.217120,0.002052,0.562500,2.590737,0.001260,1.789441
2,(todayilearned),(IAmA),0.013818,0.217120,0.008072,0.584158,2.690491,0.005072,1.882641
3,(wikipedia),(IAmA),0.004378,0.217120,0.002280,0.520833,2.398831,0.001330,1.633837
4,(bestof),(IAmA),0.002782,0.217120,0.001687,0.606557,2.793655,0.001083,1.989821
...,...,...,...,...,...,...,...,...,...
102302,"(politics, pics, science, funny, programming, ...","(reddit.com, technology, WTF)",0.002417,0.017968,0.001323,0.547170,30.452639,0.001279,2.168654
102303,"(politics, science, funny, programming, worldn...","(reddit.com, pics, technology)",0.002371,0.017785,0.001323,0.557692,31.356607,0.001280,2.220659
102304,"(politics, pics, funny, programming, worldnews...","(reddit.com, technology, science)",0.002645,0.014867,0.001323,0.500000,33.631902,0.001283,1.970266
102305,"(politics, technology, funny, programming, wor...","(pics, reddit.com, science, WTF)",0.002326,0.021707,0.001323,0.568627,26.195090,0.001272,2.267860


In [7]:
for column in ["support", "confidence", "lift", "leverage", "conviction"]:
#     rules[column] = (rules[column] - rules[column].mean())/rules[column].std()
    rules[column] = (rules[column] - rules[column].min())/(rules[column].max() - rules[column].min())

In [8]:
# OBVIOUS_SUBREDDITS = {'reddit.com', 'AskReddit', 'pics', 'WTF', 'IAmA'}

In [9]:
def get_rules(set_of_subreddits):
    antecedents_rule =  rules['antecedents'].apply(lambda x: set_of_subreddits.issuperset(x))
    new_rules =  rules[antecedents_rule].copy()
#     new_rules["Coefficient"] = new_rules["confidence"] + new_rules["support"]
#     new_rules["Coefficient"] = new_rules["lift"]
    new_rules["Coefficient"] = new_rules["confidence"] - new_rules["support"] + new_rules["lift"] + new_rules["leverage"]+ new_rules["conviction"]
#     new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits - OBVIOUS_SUBREDDITS)
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits)
    new_rules = new_rules[new_rules["consequents"].apply(lambda x: len(x) > 0)]
    if len(new_rules) == 0:
        return []
    new_rules = new_rules[['consequents', "Coefficient"]]
    new_rules = new_rules.explode("consequents")
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x:list(x)[0])
    new_rules = new_rules.groupby("consequents")["Coefficient"].max().reset_index()    
    return list(new_rules.nlargest(100, "Coefficient")["consequents"])

In [11]:
df = get_rules({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof', 'offbeat'})
df

['pics',
 'reddit.com',
 'AskReddit',
 'IAmA',
 'programming',
 'science',
 'politics']